In [11]:
# import libraries
import pygame
import random
import cv2
import mediapipe as mp
import numpy as np
import time

# define camera : 0 = camera onboard ; 1 = external camera
camera_id = 0

# define colors for blocks
colors = [
    (0, 0, 0),                      # zwart
    (0, 0, 0),                      # zwart
    (0, 202, 250),                  # blauw
    (230, 0, 126),                  # roze
    (255, 255, 0),                  # geel
]

# class figure
class Figure:
    x = 0
    y = 0

    figures = [
        [[1, 5, 9, 13], [4, 5, 6, 7]],
        [[4, 5, 9, 10], [2, 6, 5, 9]],
        [[6, 7, 9, 10], [1, 5, 6, 10]],
        [[1, 2, 5, 9], [0, 4, 5, 6], [1, 5, 9, 8], [4, 5, 6, 10]],
        [[1, 2, 6, 10], [5, 6, 7, 9], [2, 6, 10, 11], [3, 5, 6, 7]],
        [[1, 4, 5, 6], [1, 4, 5, 9], [4, 5, 6, 9], [1, 5, 6, 9]],
        [[1, 2, 5, 6]],
    ]

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = random.randint(0, len(self.figures) - 1)
        self.color = random.randint(1, len(colors) - 1)
        self.rotation = 0

    def image(self):
        return self.figures[self.type][self.rotation]

    def rotate(self):
        self.rotation = (self.rotation + 1) % len(self.figures[self.type])

# class tetris
class Tetris:
    def __init__(self, height, width):
        self.level = 2
        self.score = 0
        self.state = "start"
        self.field = []
        self.height = 0
        self.width = 0
        self.x = 750                            # binnen window x-positie origineel 100
        self.y = 60                             # binnen window y-positie origineel 60
        self.zoom = 25                          # grootte van de blokjes, origineel 20
        self.figure = None
    
        self.height = height
        self.width = width
        self.field = []
        self.score = 0
        self.state = "start"
        for i in range(height):
            new_line = []
            for j in range(width):
                new_line.append(0)
            self.field.append(new_line)

    def new_figure(self):
        self.figure = Figure(3, 0)                      # 3 is de x positie voor het blokje, 0 is de y positie voor het blokje

    def intersects(self):
        intersection = False
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    if i + self.figure.y > self.height - 1 or \
                            j + self.figure.x > self.width - 1 or \
                            j + self.figure.x < 0 or \
                            self.field[i + self.figure.y][j + self.figure.x] > 0:
                        intersection = True
        return intersection

    def break_lines(self):
        lines = 0
        for i in range(1, self.height):
            zeros = 0
            for j in range(self.width):
                if self.field[i][j] == 0:
                    zeros += 1
            if zeros == 0:
                lines += 1
                for i1 in range(i, 1, -1):
                    for j in range(self.width):
                        self.field[i1][j] = self.field[i1 - 1][j]
        self.score += lines ** 2

    def go_space(self):
        while not self.intersects():
            self.figure.y += 1
        self.figure.y -= 1
        self.freeze()

    def go_down(self):
        self.figure.y += 1
        if self.intersects():
            self.figure.y -= 1
            self.freeze()

    def freeze(self):
        for i in range(4):
            for j in range(4):
                if i * 4 + j in self.figure.image():
                    self.field[i + self.figure.y][j + self.figure.x] = self.figure.color
        self.break_lines()
        self.new_figure()
        if self.intersects():
            self.state = "gameover"

    def go_side(self, dx):
        old_x = self.figure.x
        self.figure.x += dx
        if self.intersects():
            self.figure.x = old_x

    def rotate(self):
        old_rotation = self.figure.rotation
        self.figure.rotate()
        if self.intersects():
            self.figure.rotation = old_rotation

# define teksten bij scores
textscores = ["Not bad at all",
              "Awesome", 
              "Keep on going",
              "You're a master",
              "Good job",
              "OMG",
              "GOAT!",
              "Congratz",
              "Sensational!",
              "Amazing",
              "Excellent",
              "Going strong",
              "Exceptional!",
              "Great performance",
              "Masterpiece",
              "Outstanding!",
              "Nice performance",
              "Clever",
              "Decent",
              "Solid performance",
              "Superb skills",
              "Fabulous",
              "Fantastic",
              "Incredible",
              "Terrific"]

# Initialize the game engine
pygame.init()

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
BLUE = (0, 202, 250)
YELLOW = (255, 255, 0)
PINK = (230, 0, 126)

# define window
size = (1500, 800)                            
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Infodag")

# Loop until the user clicks the close button
done = False
clock = pygame.time.Clock()
game = Tetris(25, 10)                   # aantal blokjes in hoogte en breedte
counter = 0
pressing_down = False

# camera stuff
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(camera_id)               # 0 is de interne camera
fps2 = cap.get(cv2.CAP_PROP_FPS)
cap.set(cv2.CAP_PROP_FPS, 60)

# some variables
previous_time = time.time()
current_time = time.time()
movement = "Forward"
herbeginnen = 0
previous_score = 0
todaysbest = 0

# om het spel om de 10 seconden te versnellen, 75 is initiële speed
previous_time_speed = time.time()
current_time_speed = time.time()
fps = 75

# loop
while not done:

    # om de 10 seconden gaan we spel versnellen
    current_time_speed = time.time()
    if current_time_speed - previous_time_speed > 10:
        previous_time_speed = current_time_speed
        fps = fps - 1

    # fill screen
    screen.fill(WHITE)     

    # get video
    success, frame = cap.read()
    frame = np.rot90(frame)
    frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = face_mesh.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(cv2.flip(frame,1), cv2.COLOR_RGB2BGR)
    img_h, img_w, img_c = frame.shape
    face_3d = []
    face_2d = []

    # analyze video
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 8000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
            
            # Convert it to the NumPy array
            face_2d = np.array(face_2d, dtype=np.float64)

            # Convert it to the NumPy array
            face_3d = np.array(face_3d, dtype=np.float64)

            # The camera matrix
            focal_length = 1 * img_w

            cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]])

            # The Distance Matrix
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Solve PnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            # Get rotational matrix
            rmat, jac = cv2.Rodrigues(rot_vec)

            # Get angles
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            # Get the y rotation degree
            x = angles[0] * 360
            y = angles[1] * 360

            # See where the user's head tilting

            current_time = time.time()

            if y < -10:
                if (current_time - previous_time) > 0.1:
                        movement = "Down"
                        previous_time = current_time
                else:
                    movement = "Forward"
            elif y > 10:
                if (current_time - previous_time) > 0.75:
                        movement = "Up"
                        previous_time = current_time
                else:
                    movement = "Forward"
            elif x < -10:
                if (current_time - previous_time) > 0.4:
                        movement = "Right"
                        previous_time = current_time
                else:
                    movement = "Forward"
            elif x > 10:
                if (current_time - previous_time) > 0.4:
                        movement = "Left"
                        previous_time = current_time
                else:
                    movement = "Forward"
            else:
                movement = "Forward"

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    surf = pygame.surfarray.make_surface(frame)

    # Show the video
    screen.blit(surf, (60,60))

    if game.figure is None:
        game.new_figure()
    counter += 1
    if counter > 100000:
        counter = 0

    if counter % (fps // game.level // 2) == 0 or pressing_down:
        if game.state == "start":
            game.go_down()

    # blokje bewegen op basis van de beweging in de video
    if movement == "Left":
        game.go_side(-1)

    if movement == "Right":
        game.go_side(1)

    if movement == "Up":
        game.rotate()

    if movement == "Down":
        pressing_down = True

    if movement == "Forward":
        pressing_down = False

    for event in pygame.event.get():
        if event.type == pygame.QUIT:           # window sluiten
           done = True
    
    if event.type == 768:                       # escape
        herbeginnen = 1

    # toon raster
    for i in range(game.height):
        for j in range(game.width):
            pygame.draw.rect(screen, GRAY, [game.x + game.zoom * j, game.y + game.zoom * i, game.zoom, game.zoom], 1)
            if game.field[i][j] > 0:
                pygame.draw.rect(screen, colors[game.field[i][j]],
                                 [game.x + game.zoom * j + 1, game.y + game.zoom * i + 1, game.zoom - 2, game.zoom - 1])

    # toon vallend bloksje 
    if game.figure is not None:
        for i in range(4):
            for j in range(4):
                p = i * 4 + j
                if p in game.figure.image():
                    pygame.draw.rect(screen, colors[game.figure.color],
                                     [game.x + game.zoom * (j + game.figure.x) + 1,
                                      game.y + game.zoom * (i + game.figure.y) + 1,
                                      game.zoom - 2, game.zoom - 2])


    # info
    # game.figure.x = x positie                 bij nieuw blokje altijd 3
    # game.figure.y = y positie                 bij nieuw blokje altijd 0
    # game.figure.type = soort blokje           == variabel
    # game.figure.color = kleur blokje          == variabel
    # game.figure.rotate = rotatie van blokje   bij nieuw blokje altijd 0

    # volgend bloksje                
    #if game.figure is not None:
    #    for i in range(4):
    #        for j in range(4):
    #            p = i * 4 + j
    #            if p in game.figure.image():
    #                pygame.draw.rect(screen, colors[game.figure.color],
    #                                 [1150 + game.zoom * j + 1,
    #                                  370 + game.zoom * i + 1,
    #                                  game.zoom - 2, game.zoom - 2])
    #print(game.figure.x, game.figure.y, game.figure.type, game.figure.color, game.figure.rotate)

    # display score
    if game.state != "gameover":
        if game.score > 0:
           fonts = pygame.font.SysFont('Calibri', 35, True, False)
           text = fonts.render(str(game.score), True, PINK)
           screen.blit(text, [1405, 450])    

    # display todays best
    fonts = pygame.font.SysFont('Calibri', 25, True, False)
    today = fonts.render("Today's best    "+str(todaysbest), True, BLUE)
    screen.blit(today, [750, 695])    

    # display score text
    if game.state != "gameover":
       if game.score > 0:
           if game.score != previous_score:
               previous_score = game.score
               currenttext = fonts.render(textscores[random.randint(0, len(textscores) - 1)], True, PINK)
           screen.blit(currenttext, [1050, 450])                               

    # display penta
    penta = pygame.image.load(".\penta_small.jpg").convert()
    screen.blit(penta, (1050, 60))

    # display logo
    logo = pygame.image.load(".\logohowest_small.jpg").convert()
    screen.blit(logo, (1090, 502))

    # display play your future
    playyourfuture = pygame.image.load(".\playyourfuture_small.png").convert()
    screen.blit(playyourfuture, (48, 577))

    # display gameover
    if game.state == "gameover":
        if game.score > todaysbest:
            todaysbest = game.score
        font1 = pygame.font.SysFont('Calibri', 35, True, False)
        text_game_over = font1.render("Game Over, press ESC", True, PINK)
        screen.blit(text_game_over, [1050, 450])

    # Display the nose direction
    p1 = (int(nose_2d[1])+60, 530-int(nose_2d[0]))
    p2 = (int(nose_2d[1])+60, 530-int(nose_2d[0]))
    p3 = (int(nose_2d[1])+60, 530-int(nose_2d[0]))
    p4 = (int(nose_2d[1])+60, 530-int(nose_2d[0]))
    mycolor = WHITE

    if y < -10:                                           # down
       p2 = (int(nose_2d[1])+60, 530-int(nose_2d[0])+70)
       p3 = (int(nose_2d[1])+60-20, 530-int(nose_2d[0])+70-20)
       p4 = (int(nose_2d[1])+60+20, 530-int(nose_2d[0])+70-20)
       mycolor = PINK
    elif y > 10:                                          # up
       p2 = (int(nose_2d[1])+60, 530-int(nose_2d[0])-70)
       p3 = (int(nose_2d[1])+60-20, 530-int(nose_2d[0])-70+20)
       p4 = (int(nose_2d[1])+60+20, 530-int(nose_2d[0])-70+20)
       mycolor = YELLOW
    elif x < -10:                                          # right 
       p2 = (int(nose_2d[1])+60+70, 530-int(nose_2d[0]))
       p3 = (int(nose_2d[1])+60+70-20, 530-int(nose_2d[0])+20)
       p4 = (int(nose_2d[1])+60+70-20, 530-int(nose_2d[0])-20)
       mycolor = BLACK
    elif x > 10:                                            # left
       p2 = (int(nose_2d[1])+60-70, 530-int(nose_2d[0]))
       p3 = (int(nose_2d[1])+60-70+20, 530-int(nose_2d[0])+20)
       p4 = (int(nose_2d[1])+60-70+20, 530-int(nose_2d[0])-20)
       mycolor = BLUE
    pygame.draw.line(screen, colors[game.figure.color], p1, p2, 6)
    pygame.draw.line(screen, colors[game.figure.color], p3, p2, 6)
    pygame.draw.line(screen, colors[game.figure.color], p4, p2, 6)

    # display
    pygame.display.flip()

    clock.tick(fps)

    if herbeginnen == 1:
        game.__init__(25, 10)
        herbeginnen = 0
        fps = 75

pygame.quit()